In [1]:
import importlib
import datetime
import numpy as np
import pandas as pd
from numpy import random as npr
from modules import deepfm as custom_deepfm, utils
from sklearn.model_selection import train_test_split

importlib.reload(utils)
importlib.reload(custom_deepfm)

DeepCTR-PyTorch version 0.2.7 detected. Your version is 0.2.5.
Use `pip install -U deepctr-torch` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR-Torch/releases/tag/v0.2.7


<module 'modules.deepfm' from '/root/ws/SynEvaRec/modules/modules/deepfm.py'>

In [2]:
SEED = 2021
MOVIELENS_DATASET_PATH = "data/data.csv"
REAL_DATASET_SAMPLE_SIZE = 1
CACHE_DIR = "./cache"
SYNTHETIC_USERS_COUNT = 1000
SYNTHETIC_ITEMS_COUNT = 1000
SAMPLE_SIZES = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
ALPHA_SAMPLE_RATE = 10
CTGAN_EPOCHS = 10
PART_RATIO = 0.2
N_CHUNKS = 10 

npr.seed(SEED)

In [22]:
real_dataset = pd.read_csv(MOVIELENS_DATASET_PATH)
npr.seed(SEED)
real_dataset = real_dataset.sample(frac=REAL_DATASET_SAMPLE_SIZE)
real_dataset["user_id"] = real_dataset["user_id"].astype('category').cat.codes
real_dataset["item_id"] = real_dataset["movie_id"].astype('category').cat.codes
real_dataset["rating"] = real_dataset["rating"]
real_dataset["timestamp"] = pd.to_datetime(real_dataset["timestamp"], unit="s")
real_dataset = real_dataset.sort_values("timestamp")
real_dataset = real_dataset.drop(["movie_name", "timestamp", "item"], axis=1)
real_dataset

/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,movie_id,rating,tag0,tag1,tag2,tag3,tag4,tag5,tag6,...,tag13,tag14,tag15,tag16,tag17,gender,age,occupation,zip,item_id
456790,6039,858,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,802
456732,6039,2384,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,M,25,6,11106,2191
456672,6039,593,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,579
456641,6039,1961,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,1781
456842,6039,2019,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,25,6,11106,1839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373230,4957,2399,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,18,7,55403,2206
373293,4957,1407,5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,M,18,7,55403,1305
373229,4957,3264,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,M,18,7,55403,3040
373382,4957,2634,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,M,18,7,55403,2432


In [23]:
for i in range(18):
    real_dataset[f"tag{i}"] = real_dataset[f"tag{i}"].astype("category")

In [24]:
fields = [
    "gender",
    "occupation",
    "zip",
    "age",
    "user_id",
    "item_id",
    "movie_id",
    "rating",
    *[f"tag{i}" for i in range(18)]
]

for field in fields:
    real_dataset[field] = real_dataset[field].astype("category")


In [25]:
real_dataset.dtypes

user_id       category
movie_id      category
rating        category
tag0          category
tag1          category
tag2          category
tag3          category
tag4          category
tag5          category
tag6          category
tag7          category
tag8          category
tag9          category
tag10         category
tag11         category
tag12         category
tag13         category
tag14         category
tag15         category
tag16         category
tag17         category
gender        category
age           category
occupation    category
zip           category
item_id       category
dtype: object

In [26]:
y = real_dataset["rating"]
X = real_dataset.drop(["user_id", "item_id", "movie_id", "zip", "rating"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
X

,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,...,tag11,tag12,tag13,tag14,tag15,tag16,tag17,gender,age,occupation
456790,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6
456732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6
456672,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6
456641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6
456842,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373230,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,M,18,7
373293,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M,18,7
373229,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,M,18,7
373382,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M,18,7


In [28]:
from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.models.deepnets import DeepFM
from deeptables.datasets import dsutils
from tensorflow import keras
from deeptables.models import deeptable

conf = ModelConfig(
    metrics=['RootMeanSquaredError'], 
    nets=DeepFM, 
    optimizer=keras.optimizers.RMSprop(),
    auto_discrete=True
)
dt = deeptable.DeepTable(config=conf)
model, history = dt.fit(x_train, y_train, epochs=10)
score = dt.evaluate(x_test, y_test, batch_size=512, verbose=0)
score

5 class detected, inferred as a [multiclass classification] task
Preparing features taken 0.2900547981262207s
Imputation taken 19.756896257400513s
Categorical encoding taken 36.58749556541443s
Discretization taken 3.4332275390625e-05s
Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:1, mode:min
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (21)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 9, 23]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
---------------------------------------------------------
con

5002/5002 [==============================] - 35s 7ms/step - loss: 1.4362 - root_mean_squared_error: 0.3841 - val_loss: 1.4217 - val_root_mean_squared_error: 0.3828
Epoch 2/10
5001/5002 [============================>.] - ETA: 0s - loss: 1.4233 - root_mean_squared_error: 0.3830WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 36s 7ms/step - loss: 1.4233 - root_mean_squared_error: 0.3830 - val_loss: 1.4198 - val_root_mean_squared_error: 0.3826
Epoch 3/10
4993/5002 [============================>.] - ETA: 0s - loss: 1.4211 - root_mean_squared_error: 0.3828WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 33s 7ms/step - loss: 1.4212 - root_mean_squared_error: 0.3828 - val_loss: 1.4168 - val_root_mean_squared_error: 0.3823
Epoch 4/10
4996/5002 [============================>.] - ETA: 0s - loss: 1.4197 - root_mean_squared_error: 0.3826WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 35s 7ms/step - loss: 1.4197 - root_mean_squared_error: 0.3826 - val_loss: 1.4155 - val_root_mean_squared_error: 0.3822
Epoch 5/10
4999/5002 [============================>.] - ETA: 0s - loss: 1.4189 - root_mean_squared_error: 0.3825WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


...

5002/5002 [==============================] - 35s 7ms/step - loss: 1.4176 - root_mean_squared_error: 0.3824 - val_loss: 1.4136 - val_root_mean_squared_error: 0.3820
Epoch 8/10
4996/5002 [============================>.] - ETA: 0s - loss: 1.4173 - root_mean_squared_error: 0.3824WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 36s 7ms/step - loss: 1.4173 - root_mean_squared_error: 0.3824 - val_loss: 1.4141 - val_root_mean_squared_error: 0.3820
Epoch 9/10
4996/5002 [============================>.] - ETA: 0s - loss: 1.4170 - root_mean_squared_error: 0.3824WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 36s 7ms/step - loss: 1.4170 - root_mean_squared_error: 0.3824 - val_loss: 1.4131 - val_root_mean_squared_error: 0.3820
Epoch 10/10
4994/5002 [============================>.] - ETA: 0s - loss: 1.4164 - root_mean_squared_error: 0.3823WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


5002/5002 [==============================] - 35s 7ms/step - loss: 1.4164 - root_mean_squared_error: 0.3823 - val_loss: 1.4146 - val_root_mean_squared_error: 0.3821
Model has been saved to:dt_output/dt_20210826 133250_fm_nets_linear_dnn_nets/fm_nets+linear+dnn_nets.h5


{'loss': 1.4144153594970703, 'root_mean_squared_error': 0.3820882737636566}

In [9]:
res = dt.predict(x_test)

In [10]:
res

array([4, 3, 4, ..., 5, 3, 3])

In [51]:
from sdv.tabular import CTGAN


def fit_syn_generator(df):
    df = df[
        ["user_id", "item_id", "age", "occupation", "gender", *[f"tag{n}" for n in range(18)]]
    ]
    df = df.sample(frac=0.05)
    
    model = CTGAN(verbose=True, epochs=CTGAN_EPOCHS)
    df = df.astype("int64", errors="ignore") # Convert all numbers to int64
    df["gender"] = df["gender"].astype("object")
    print(df.dtypes)
    model.fit(df.copy())
    return model

## Constraints

In [14]:
from sdv.constraints import CustomConstraint


def is_gender_valid(column_data):
    return set(column_data).unique() == {"M", "N"}


def is_tag_valid(column_data):
    return set(column_data) == {0, 1}


constraints = [
    CustomConstraint(columns=[f"tag{n}" for n in range(18)], is_valid=is_tag_valid),
    CustomConstraint(columns="gender", is_valid=is_gender_valid)
]

TypeError: __init__() got an unexpected keyword argument 'columns'

In [52]:
%%time
generator = fit_syn_generator(real_dataset)

user_id        int64
item_id        int64
age            int64
occupation     int64
gender        object
tag0           int64
tag1           int64
tag2           int64
tag3           int64
tag4           int64
tag5           int64
tag6           int64
tag7           int64
tag8           int64
tag9           int64
tag10          int64
tag11          int64
tag12          int64
tag13          int64
tag14          int64
tag15          int64
tag16          int64
tag17          int64
dtype: object


/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/p

/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TV

Epoch 1, Loss G: -1.1923,Loss D:  0.1394
Epoch 2, Loss G: -1.5521,Loss D:  0.1550
Epoch 3, Loss G: -1.3295,Loss D:  0.1461
Epoch 4, Loss G: -2.2736,Loss D:  0.2031
Epoch 5, Loss G: -1.6773,Loss D: -0.1678
Epoch 6, Loss G: -2.3650,Loss D:  0.1369
Epoch 7, Loss G: -1.9743,Loss D:  0.1058
Epoch 8, Loss G: -1.5303,Loss D:  0.1653
Epoch 9, Loss G: -2.0822,Loss D: -0.2555
Epoch 10, Loss G: -1.8577,Loss D: -0.1694
CPU times: user 17min 7s, sys: 10.5 s, total: 17min 18s
Wall time: 24min 21s


In [53]:
syn_sample = generator.sample(int(len(real_dataset) * 0.05))
syn_sample

,user_id,item_id,age,occupation,gender,tag0,tag1,tag2,tag3,tag4,...,tag8,tag9,tag10,tag11,tag12,tag13,tag14,tag15,tag16,tag17
0,5109,2472,35,4,F,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4930,2271,24,3,M,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5119,-6,24,7,M,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,4817,1585,25,0,F,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5828,2874,34,4,M,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50005,1970,928,18,3,F,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
50006,1747,2591,49,14,M,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
50007,2457,890,24,1,F,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50008,5985,1518,56,16,M,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
syn_data = syn_sample.copy()
syn_sample["user_id"] = syn_sample.user_id.astype("category").cat.codes
syn_sample["item_id"] = syn_sample.item_id.astype("category").cat.codes
syn_sample

,user_id,item_id,age,occupation,gender,tag0,tag1,tag2,tag3,tag4,...,tag8,tag9,tag10,tag11,tag12,tag13,tag14,tag15,tag16,tag17
0,5111,2644,35,4,F,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4932,2443,24,3,M,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5121,169,24,7,M,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,4819,1757,25,0,F,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5830,3046,34,4,M,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50005,1975,1100,18,3,F,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
50006,1752,2763,49,14,M,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
50007,2462,1062,24,1,F,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50008,5987,1690,56,16,M,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
fields = [
    "gender",
    "occupation",
    "age",
    *[f"tag{i}" for i in range(18)]
]

for field in fields:
    syn_sample[field] = syn_sample[field].astype("category")


In [65]:
ids = syn_sample[["user_id", "item_id"]]

In [66]:
syn_sample = syn_sample[x_train.columns]

In [67]:
syn_sample

,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,...,tag11,tag12,tag13,tag14,tag15,tag16,tag17,gender,age,occupation
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,F,35,4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M,24,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,M,24,7
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,F,25,0
4,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,M,34,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50005,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,F,18,3
50006,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,M,49,14
50007,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,F,24,1
50008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M,56,16


In [68]:
x_train.dtypes

tag0          category
tag1          category
tag2          category
tag3          category
tag4          category
tag5          category
tag6          category
tag7          category
tag8          category
tag9          category
tag10         category
tag11         category
tag12         category
tag13         category
tag14         category
tag15         category
tag16         category
tag17         category
gender        category
age           category
occupation    category
dtype: object

In [69]:
res = dt.predict(syn_sample)

In [70]:
res

array([4, 4, 4, ..., 4, 4, 4])

In [72]:
df = syn_sample.copy()
df["rating"] = res
df["user_id"] = ids["user_id"]
df["item_id"] = ids["item_id"]

In [73]:
df

,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,...,tag14,tag15,tag16,tag17,gender,age,occupation,rating,user_id,item_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,F,35,4,4,5111,2644
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,M,24,3,4,4932,2443
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,M,24,7,4,5121,169
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,F,25,0,4,4819,1757
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,M,34,4,4,5830,3046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,F,18,3,4,1975,1100
50006,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,M,49,14,4,1752,2763
50007,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,F,24,1,4,2462,1062
50008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,M,56,16,4,5987,1690


In [74]:
val = {}
for v in res:
    if v in val:
        val[v] += 1
    else:
        val[v] = 0
val

{4: 49953, 5: 55}

In [75]:
val = {}
for v in y:
    if v in val:
        val[v] += 1
    else:
        val[v] = 0
val

{4: 348970, 5: 226309, 3: 261196, 2: 107556, 1: 56173}

In [76]:
eval_df = df[["user_id", "item_id", "rating"]]
eval_df

,user_id,item_id,rating
0,5111,2644,4
1,4932,2443,4
2,5121,169,4
3,4819,1757,4
4,5830,3046,4
...,...,...,...
50005,1975,1100,4
50006,1752,2763,4
50007,2462,1062,4
50008,5987,1690,4


In [77]:
def evaluate(df):
    errors = {
        "svd": utils.evaluate_svd(df.copy(), rating_scale=(0, 5)),
        "knn": utils.evaluate_knn(df.copy(), rating_scale=(0, 5)),
        "autorec": utils.evaluate_autorec(df.copy())
    }
    return errors

In [78]:
eval_result = evaluate(eval_df)

RMSE: 0.1022
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0376
Load data finished. Number of users: 6307 Number of items: 3787
IAutoRec.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Network built
Epoch: 0000; RMSE:2.709767241072753; MAE:2.6821474808364174
Epoch: 0003; RMSE:0.9093598293336618; MAE:0.6784414506345056
Epoch: 0006; RMSE:0.5332048513213612; MAE:0.3859196398045402
Epoch: 0009; RMSE:0.3885062410055412; MAE:0.19293089714520611
Epoch: 0012; RMSE:0.36290974649484736; MAE:0.15001341318180786
Epoch: 0015; RMSE:0.3530997592959878; MAE:0.12841742674444104
Epoch: 0018; RMSE:0.3480190619857909; MAE:0.11775628825108142
Epoch: 0021; RMSE:0.34434337030007967; MAE:0.10929110357846948
Epoch: 0024; RMSE:0.34164509893789724; MAE:0.1034850406889047
Epoch: 0027; RMSE:0.34018331088451226; MAE:0.09994716773244727
Epoch: 0030; RMSE:0.3382447900758268; MAE:0.0955708137321868
Epoch: 0033; RMSE:0.33727667146068524; MAE:0.09228385654057783
Epoch: 0036; RMSE:0.33690058083379654; MAE:0.09029749259121017
Epoch: 0039; RMSE:0.3361923737765595; MAE:0.0880576446065039
Epoch: 0042; RMSE:0.33535896404620874; MAE:0.08641074199315626
Epoch: 0045; RMSE:0.33541581212489735; MAE:0.08579486674

Epoch: 0414; RMSE:0.3509154194740815; MAE:0.12016909162720113
Epoch: 0417; RMSE:0.35079260775010485; MAE:0.11731107480801292
Epoch: 0420; RMSE:0.34766428893148316; MAE:0.1125686021143978
Epoch: 0423; RMSE:0.3528528951646531; MAE:0.13036001732519945
Epoch: 0426; RMSE:0.3506237947895351; MAE:0.1221156822184159
Epoch: 0429; RMSE:0.35010451540421544; MAE:0.11991083756992593
Epoch: 0432; RMSE:0.35060223883095804; MAE:0.11946690675103455
Epoch: 0435; RMSE:0.35281007890705146; MAE:0.12668233259580594
Epoch: 0438; RMSE:0.35167412257154046; MAE:0.1220657927794411
Epoch: 0441; RMSE:0.3490781624125887; MAE:0.11801187677987618
Epoch: 0444; RMSE:0.35200464890476385; MAE:0.12160389637915359
Epoch: 0447; RMSE:0.35441204385252006; MAE:0.12857370057400258
Epoch: 0450; RMSE:0.34920448218447114; MAE:0.11841965316492133
Epoch: 0453; RMSE:0.35014197188873003; MAE:0.11974435114962106
Epoch: 0456; RMSE:0.351391203158144; MAE:0.11835950666191784
Epoch: 0459; RMSE:0.3484164402796324; MAE:0.11604763876053621
Ep